In [7]:
from pathlib import Path

import eelbrain
import mne
from scipy.io import loadmat
import numpy as np



# Data locations
DATA_ROOT = Path("~").expanduser() / 'Data' / 'cocoha'
DATA_PREPROC = DATA_ROOT / "data_preprocessed"



In [17]:
for file in DATA_PREPROC.glob("*.mat"):

    SUBJECT = file.stem.split("_")[0]

    dst_dir = DATA_ROOT / 'eeg' / SUBJECT
    dst_dir.mkdir(exist_ok=True, parents=True)
    eeg_dst = dst_dir / f'{SUBJECT}_eeg.pickle'
    padded_eeg_dst = dst_dir / f'{SUBJECT}_padded_eeg.pickle'
    if eeg_dst.exists() and padded_eeg_dst.exists():
        print(f"Files for {SUBJECT} already exist, skipping.")
        continue


    print("Saving ~", SUBJECT, "...")

    mat = loadmat(file, squeeze_me=True, struct_as_record=False)
    data = mat['data']

    # --- get channel names ---
    ch_names = data.dim.chan.eeg[0][:64].tolist()

    # --- collect all trials ---
    eeg_trials = []
    padded_eeg_trials = []

    for trial in data.eeg:  # 60 trials
        eeg = trial[:, :64]        # remove extra channels

        # Create time axis
        time_axis = eelbrain.UTS(0, 1 / data.fsample.eeg, len(eeg))  # fsample rate is 64 Hz, so time step is 1/64 seconds

        # Create Sensor axis
        sensor_axis = eelbrain.Categorial(values=ch_names[:64], name='sensor')

        # Save the eeg data as an NDVar with dimensions (time, sensor)
        eeg = eelbrain.NDVar(eeg, dims=(time_axis, sensor_axis), name='eeg')

        # Save the padded eeg data as an NDVar with dimensions (time, sensor)
        padded_eeg = eelbrain.pad(eeg, tstart=-0.100, tstop=eeg.time[-1] + 1, name='padded_eeg')


        eeg_trials.append(eeg)
        padded_eeg_trials.append(padded_eeg)

    # --- concatenate trials ---
    eeg_concat = eelbrain.concatenate(eeg_trials)
    padded_eeg_concat = eelbrain.concatenate(padded_eeg_trials)

    eelbrain.save.pickle(eeg_concat, eeg_dst)
    eelbrain.save.pickle(padded_eeg_concat, padded_eeg_dst)
    print(f"Saved eeg and padded eeg for {SUBJECT}")


Saving ~ S5 ...
Saved eeg and padded eeg for S5
Saving ~ S15 ...
Saved eeg and padded eeg for S15
Saving ~ S11 ...
Saved eeg and padded eeg for S11
Saving ~ S1 ...
Saved eeg and padded eeg for S1
Saving ~ S4 ...
Saved eeg and padded eeg for S4
Saving ~ S14 ...
Saved eeg and padded eeg for S14
Saving ~ S10 ...
Saved eeg and padded eeg for S10
Saving ~ S17 ...
Saved eeg and padded eeg for S17
Saving ~ S7 ...
Saved eeg and padded eeg for S7
Saving ~ S18 ...
Saved eeg and padded eeg for S18
Saving ~ S3 ...
Saved eeg and padded eeg for S3
Saving ~ S13 ...
Saved eeg and padded eeg for S13
Saving ~ S8 ...
Saved eeg and padded eeg for S8
Saving ~ S16 ...
Saved eeg and padded eeg for S16
Saving ~ S6 ...
Saved eeg and padded eeg for S6
Saving ~ S2 ...
Saved eeg and padded eeg for S2
Saving ~ S9 ...
Saved eeg and padded eeg for S9
Saving ~ S12 ...
Saved eeg and padded eeg for S12


In [19]:
# Sanity check: load one of the saved files and print its shape
eeg = eelbrain.load.unpickle(DATA_ROOT / 'eeg' / "S1" / "S1_eeg.pickle")
padded_eeg = eelbrain.load.unpickle(DATA_ROOT / 'eeg' / "S1" / "S1_padded_eeg.pickle")
print(eeg, padded_eeg)

<NDVar 'eeg': 192000 time, 64 sensor> <NDVar 'padded_eeg': 196200 time, 64 sensor>
